In [ ]:
# torch version은 1.4으로 맞춰서 실행해야합니다. 
!conda install -y pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch

In [ ]:
# torch version은 1.4으로 맞춰서 실행해야합니다. 
import torch
torch.__version__

In [ ]:
import albumentations as A
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import pandas as pd
import numpy as np
import cv2
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, root, phase='train', img_cropped = None, label_cropped = None):
        self.root = root
        self.phase = phase
        self.data = {}
                    
        # 마찬가지로 TEST 데이터의 경우에도 이미지를 늘리고 다시 줄인 듯
        # 해당 부분에 대해서 TRAIN과 마찬가지로 CROP해서 결합해도 괜찮을 듯 
        if phase == 'test':
            img_cropped = []
            for i in range(238):
                img = cv2.imread(self.root + f'/test/{i}.png')
                img_cropped.append(img)
        
        self.data['input'] = img_cropped
        self.data['output'] = label_cropped            
        
    def __getitem__(self, index):
        # 실험해야할 부분 
        # Augmentation으로 수정해야함 
        transform_tr = transforms.Compose([
                transforms.ToTensor(), 
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                ])

        transform_te = transforms.Compose([
                transforms.ToTensor()
                ])       
        
        img = self.data['input'][index]        
        input_img = transform_tr(img)
                
        if (self.phase == 'train') | (self.phase == 'valid'):
            mask = self.data['output'][index]
            output_mask = transform_te(mask)
            return (input_img, output_mask)
        else:
            dummy = []
            return (input_img, dummy)
    
    def __len__(self):
        return len(self.data['input'])
    
    def get_label_file(self):
        return self.label_path

def data_loader(root, phase='train', batch_size=4, img_cropped = None, label_cropped = None):
    if phase == 'train':
        shuffle = True
    else:
        shuffle = False
    
    dataset = CustomDataset(root, phase, img_cropped = img_cropped, label_cropped = label_cropped)
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [ ]:
# -*- coding: utf-8 -*- 

import os
import math
import datetime
import numpy as np
import pandas as pd
import time
import torchvision
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import argparse

DATASET_PATH = '/home/workspace/data/.train/.task142/'


def _infer(model, cuda, data_loader):
    pred = []
    for idx, (img, _) in enumerate(data_loader):
        if cuda:
            img = img.cuda()

        output = model(img)
        output = output['out'].detach().cpu().numpy()
        output = output.argmax(1)
        pred.extend(output)

    return pred


def feed_infer(output_file, infer_func, fold):
    prediction = infer_func()
    print('write output')
    out_pred = pd.DataFrame()
    for pred in prediction:
        pred = pd.DataFrame(pred)
        out_pred = pd.concat([out_pred, pred], axis=0)
    out_pred = out_pred + 1
    out_pred.to_csv('/home/workspace/user-workspace/prediction/prediction{}.csv'.format(fold), index=None)
    
    if os.stat(output_file).st_size == 0:
        raise AssertionError('output result of inference is nothing')


def test(prediction_file_name, model, test_dataloader, cuda, fold):
    feed_infer(prediction_file, lambda : _infer(model, cuda, data_loader=test_dataloader), fold=fold)


def save_model(model_name, model, optimizer, scheduler, fold):
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }
    torch.save(state, 'deep_resnet101_pixel50_' + fold + '.pth')
    print('model saved')


def load_model(model_name, model, optimizer=None, scheduler=None):
    state = torch.load(os.path.join(model_name))
    model.load_state_dict(state['model'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    if scheduler is not None:
        scheduler.load_state_dict(state['scheduler'])
    print('model loaded')
    return model

In [ ]:
img_cropped = []
label_cropped = []

root = '/home/workspace/data/.train/.task142/'

data_path=os.path.join(root, 'train.png')
label_path=os.path.join(root, 'train_label.png')
img = cv2.imread(data_path)

# class_mask가 1부터 시작하므로, 0부터 시작하도록 세팅합니다. 
class_mask = np.array(pd.read_csv(root + 'class_mask.csv'), dtype=np.int) - 1
class_mask = to_categorical(class_mask, 9)

for i in tqdm(range(0, 8750-512, 512)):
    for j in range(0, 7225-512, 512):
        img_cropped.append(img[i:i+512, j:j+512].copy())
        label_cropped.append(class_mask[i:i+512, j:j+512].copy())

In [ ]:
# validation의 정확한 점수를 체크하기 위해서 pixel accuracy 계산 
def pixel_accuracy(pred, sol):
    pred = pred.argmax(axis=1)
    sol = sol.argmax(axis=1)
    
    acc_sum = np.sum((pred == sol))
    acc = float(acc_sum / (512 * 512 + 1e-10))
    
    return acc

In [ ]:
# 시드의 고정을 위해서 seed 세팅 
import random
import gc
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
# 파라미터 고정 
base_lr = 0.003
cuda = True
num_epochs = 50
print_iter = 50
model_name = "deep_resnet101"
prediction_file = "prediction.csv"
batch = 8
mode = "train"

In [ ]:
seed_everything(0)

In [ ]:
# 5폴드 분할 후 학습 진행 
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
kf = KFold(5, shuffle=True, random_state=0)
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# 데이터를 Numpy 형태로 변환 
img_cropped = np.array(img_cropped)
label_cropped = np.array(label_cropped)
for fold, (tr_idx, val_idx) in enumerate(kf.split(range(0, len(img_cropped)))):        
    print("#"*50, fold, "#"*50)
    # 데이터셋 로드 
    train_dataloader = data_loader(root=DATASET_PATH, phase='train', batch_size=batch, img_cropped = img_cropped[tr_idx], label_cropped = label_cropped[tr_idx])
    valid_dataloader = data_loader(root=DATASET_PATH, phase='valid', batch_size=batch, img_cropped = img_cropped[val_idx], label_cropped = label_cropped[val_idx])

    time_ = datetime.datetime.now()
    num_batches = len(train_dataloader)
    best_valid = 0
    # Deeplabv3 모델 불러오기 
    model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=True, num_classes=9).cuda()
    model = model.cuda()
    
    # 로스 함수 설정 
    loss_fn = nn.BCEWithLogitsLoss(reduction = 'sum')
    loss_fn = loss_fn.cuda()
    
    # optimizer, scheduler 함수 설정 
    optimizer = Adam([param for param in model.parameters() if param.requires_grad],lr=base_lr, weight_decay=1e-4)
    scheduler = StepLR(optimizer, step_size=40, gamma=0.1)
    
    # 50번의 Epoch를 반복 
    for epoch in range(num_epochs):
        train_loss, valid_loss, train_pixel ,valid_pixel  = 0, 0, 0, 0
        model.train()
        for iter_, (i, l) in enumerate(train_dataloader):
            i = i.cuda()
            l = l.cuda()
            out = model(i)
            loss = loss_fn(out['out'].float(), l.float())
            train_pixel += pixel_accuracy(out['out'].float().cpu().detach().numpy(), l.float().cpu().detach().numpy()) / len(train_dataloader.dataset.data['input'])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() / (512*512*9*len(train_dataloader.dataset.data['input']))
            del loss; del i; del l; del out; gc.collect()
            
        model.eval()
        for iter_, (i, l) in enumerate(valid_dataloader):
            i = i.cuda()
            l = l.cuda()
            out = model(i)
            loss = loss_fn(out['out'].float(), l.float())
            valid_pixel += pixel_accuracy(out['out'].float().cpu().detach().numpy(), l.float().cpu().detach().numpy()) / len(valid_dataloader.dataset.data['input'])
            valid_loss += loss.item() / (512*512*9*len(valid_dataloader.dataset.data['input']))
            del loss; del i; del l; del out; gc.collect()
            
        
        if (iter_ + 1) % 1 == 0:
            elapsed = datetime.datetime.now() - time_
            print('[{:.0f}/{:d}] train loss({:.4f}) valid loss({:.4f}) train pixel Accuracy({:.4f}) valid pixel Accuracy({:.4f})'.format(
                      int(epoch), num_epochs, train_loss, valid_loss, train_pixel, valid_pixel))
            time_ = datetime.datetime.now()

        # scheduler update
        scheduler.step()
        
        # 가장 좋은 모델을 Valid Pixel Accuracy를 기준으로 선택 
        if best_valid < valid_pixel: 
            best_valid = valid_pixel
            # save model
            save_model(str(epoch + 1), model, optimizer, scheduler, str(fold))

In [ ]:
# 학습과정과 추론과정을 분리하기 위해서, 학습된 파라미터를 불러와서 실행 
## 참고로 여기에서는 Torch 1.6버전을 사용해도 무방했습니다. 
import gc
mode = "test"
root = '/home/workspace/data/.train/.task142/'

batch = 1

preds = []
for i in range(5):
    # 저장된 모델의 가중치를 불러와서 로드
    model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=True, num_classes=9).cuda()
    model = load_model('deep_resnet101_pixel50_{}.pth'.format(i), model)
    
    model.eval()
    # get data loader
    test_dataloader = data_loader(root=DATASET_PATH, phase=mode, batch_size=batch)
    pred = []
    # 데이터를 불러와서 예측 
    for idx, (img, _) in enumerate(test_dataloader):
        img = img.cuda()
        output = model(img)
        output1 = output['out'].data.detach().cpu().numpy()
        pred.extend(output1)
    preds.append(pred)
    
    model = None
    del model, output, pred; gc.collect()
    torch.cuda.empty_cache()
del test_dataloader

## 예측과정에서 메모리 이슈가 발생하는 경우가 많아서 파일을 저장해서 불러오는 식으로 사용
np.save('preds(5)', preds)

In [ ]:
preds = np.load('preds(5).npy')

In [ ]:
# 5폴드 앙상블을 진행 
print('fold average')
final_answer = np.mean(np.array(preds), axis=0)
final_answer = final_answer.argmax(axis=1)

# 기존의 저장코드의 속도가 오래 걸려서, 예측된 결과 하나씩 바로 저장 
import csv
print('write output')
out_pred = pd.DataFrame()
with open('/home/workspace/user-workspace/prediction/prediction(tree50_deep).csv','w') as f:
    writer = csv.writer(f, delimiter = ',')
    writer.writerow([c for c in range(512)])
    for pred in tqdm(final_answer):
        pred = pd.DataFrame(pred)
        pred += 1
        for idx in range(512):
            writer.writerow(pred.loc[idx])
    del pred

In [ ]:
# 최종 제출 
import os
from nipa.taskSubmit import nipa_submit

team_id = "1192"
task_no= "142"
prediction_path = '/home/workspace/user-workspace/prediction/prediction(tree50_deep).csv'

# 파일 존재 여부 확인
print("is file: ", os.path.isfile(prediction_path))

# 제출 성공
nipa_submit(team_id=team_id,
task_no=task_no,
result=prediction_path
)

## TTA 적용시 

In [ ]:
def torch_tta(x):
    preds = torch.rot90(model(torch.rot90(x, 1, [2, 3]))['out'], -1, [2, 3])
    preds += torch.rot90(model(torch.rot90(x, 2, [2, 3]))['out'], -2, [2, 3])
    preds += torch.rot90(model(torch.rot90(x, 3, [2, 3]))['out'], -3, [2, 3])
    preds += torch.flip(model(torch.flip(x, [2]))['out'], [2])
    preds += torch.flip(model(torch.flip(x, [3]))['out'], [3])
    preds += model(x)['out']
    preds /= 6
    return preds

In [ ]:
import gc
mode = "test"
root = '/home/workspace/data/.train/.task142/'

batch = 1

preds = []
for i in range(5):
    model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=True, num_classes=9).cuda()
    model = load_model('deep_resnet101_pixel75_{}.pth'.format(i), model)
    
    model.eval()
    # get data loader
    test_dataloader = data_loader(root=DATASET_PATH, phase=mode, batch_size=batch)
    pred = []
    for idx, (img, _) in enumerate(test_dataloader):
        img = img.cuda()
        output = torch_tta(img)
        output1 = output.data.detach().cpu().numpy()
        pred.extend(output1)
    preds.append(pred)
    
    model = None
    del model, output, pred; gc.collect()
    torch.cuda.empty_cache()
del test_dataloader
np.save('preds(tta)', preds)

In [ ]:
# mean을 취하고 argmax를 하는게 잘못된 것일까?
## argmax 이후에 median을 취하는게 나을 수도 있음 
print('fold average')
final_answer = np.mean(np.array(preds), axis=0)
final_answer = final_answer.argmax(axis=1)

import csv
print('write output')
out_pred = pd.DataFrame()
with open('/home/workspace/user-workspace/prediction/prediction(tta).csv','w') as f:
    writer = csv.writer(f, delimiter = ',')
    writer.writerow([c for c in range(512)])
    for pred in tqdm(final_answer):
        pred = pd.DataFrame(pred)
        pred += 1
        for idx in range(512):
            writer.writerow(pred.loc[idx])
    del pred

In [ ]:
import os
from nipa.taskSubmit import nipa_submit

team_id = "1192"
task_no= "142"
prediction_path = '/home/workspace/user-workspace/prediction/prediction(tta).csv'

# 파일 존재 여부 확인
print("is file: ", os.path.isfile(prediction_path))

# 제출 성공
nipa_submit(team_id=team_id,
task_no=task_no,
result=prediction_path
)